In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

import os
os.chdir('/content/drive/My Drive/Colab Notebooks/refine-epitope-deep-learning')

Mounted at /content/drive/


In [ ]:
!pip install transformers
!pip install optuna

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 7.3 MB/s 
     |████████████████████████████████| 101 kB 12.2 MB/s 
     |████████████████████████████████| 6.6 MB 76.5 MB/s 
     |████████████████████████████████| 596 kB 68.8 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 308 kB 8.5 MB/s 
     |████████████████████████████████| 209 kB 32.8 MB/s 
     |████████████████████████████████| 81 kB 11.0 MB/s 
     |████████████████████████████████| 78 kB 8.5 MB/s 
     |████████████████████████████████| 112 kB 79.1 MB/s 
     |████████████████████████████████| 147 kB 76.1 MB/s 
     |████████████████████████████████| 49 kB 5.9 MB/s 
  Created wheel for py

In [ ]:
with open("iedb_linear_epitopes.fasta", 'r') as fp:
    text = fp.read().split('\n')

list_positive_negative = []

for row in text:
  if ('PositiveID' not in row) and ('NegativeID' not in row) and (len(row) < 512): 
    list_positive_negative.append(row)


file = open("roberta_iedb_linear_epitopes.txt", "w") 
for item in list_positive_negative:
  epitope= ''.join([c for c in item if c.isupper()])
  no_epitope = item.split(epitope)
  file.writelines(" ".join(no_epitope[0]) + '\n')
  file.writelines(" ".join(epitope) + '\n')
  file.writelines(" ".join(no_epitope[1]) + '\n\n')

file.close()


In [ ]:
from transformers import BertTokenizer, BertForMaskedLM, BertForPreTraining, BertForSequenceClassification
from transformers import LineByLineTextDataset
from transformers import DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments
from transformers import BertConfig, BertForPreTraining
from transformers import TextDatasetForNextSentencePrediction

In [ ]:
config = BertConfig(    # https://huggingface.co/transformers/model_doc/bert.html#bertconfig
    #vocab_size=20000, # 
    # hidden_size=512,
    # num_hidden_layers=12,    # layer num
    # num_attention_heads=8,    # transformer attention head number
    # intermediate_size=3072,   # transformer 
    # hidden_act="gelu",
    # hidden_dropout_prob=0.1,
    # attention_probs_dropout_prob=0.1,
    #max_position_embeddings=512    # embedding size 
    # type_vocab_size=2,    # token type 
    # pad_token_id=0,
    # position_embedding_type="absolute"
)

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', model_max_length=512, truncation=True)

model = BertForPreTraining(config=config)
#model = BertForPreTraining.from_pretrained('bert-base-uncased')

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
#dataset = LineByLineTextDataset(
#    tokenizer=tokenizer,
#    file_path="roberta_iedb_linear_epitopes.txt",
#    block_size=512,
#)

In [ ]:
dataset = TextDatasetForNextSentencePrediction(
    tokenizer=tokenizer,
    file_path='roberta_iedb_linear_epitopes.txt',
    block_size=512,
    overwrite_cache=False,
 #   short_seq_probability=0.1,
    nsp_probability=0.5,
)


/usr/local/lib/python3.7/dist-packages/transformers/data/datasets/language_modeling.py:366: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  FutureWarning,


In [ ]:

for index, example in enumerate(dataset.examples):
  if len(example["input_ids"]) > 512:
    #print(dataset.examples[index]["input_ids"])
    dataset.examples[index]["input_ids"] = dataset.examples[index]["input_ids"][:512]
    dataset.examples[index]["token_type_ids"] = dataset.examples[index]["token_type_ids"][:512]
    

In [ ]:
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

In [ ]:
for example in dataset.examples[0:1]:
    print(example)

{'input_ids': tensor([ 101, 1049, 1037, 1055, 1053, 1047, 1054, 1052, 1055, 1053, 1054, 1044,
        1043, 1055, 1047, 1061, 1048, 1037, 1056, 1037, 1055, 1056, 1049, 1040,
        1044, 1037, 1054, 1044, 1043, 1042, 1048, 1052, 1054, 1044, 1054, 1040,
        1056, 1043, 1045, 1048, 1040, 1055, 1045, 1043, 1054, 1042, 1042, 1043,
        1043, 1040, 1054, 1043, 1037, 1052, 1047, 1054, 1043, 1055, 1043, 1047,
        1040, 1055, 1044, 1044, 1052, 1037, 1054, 1056, 1037, 1044, 1061, 1043,
        1055, 1048, 1052, 1053,  102, 1049, 1047, 1042, 1047, 1047, 1056, 1045,
        1043, 1037, 1049, 1037, 1048, 1056, 1056, 1049, 1042, 1058, 1037, 1058,
        1055, 1037, 1055, 1037, 1058, 1041, 1047, 1050, 1045, 1056, 1058, 1056,
        1037, 1055, 1058, 1040, 1052, 1058, 1045, 1040, 1048, 1048, 1053, 1037,
        1040, 1043, 1050, 1037, 1048, 1052, 1055, 1037, 1058, 1047, 1048, 1037,
        1061, 1055, 1052, 1037, 1055, 1047, 1056, 1042, 1041, 1055, 1061, 1054,
        1058, 1049, 1056, 

In [ ]:
#print(data_collator(dataset.examples)['input_ids'][0])

In [ ]:
# m a s q k r p s q r h g s k y l a t a s t m d h a r h g f l p r h r d t g i l d s i g r f f g g d r g a p k r g s g k d s h h p a r t a h y g s l p q

tokenizer.decode(data_collator(dataset.examples)['input_ids'][0].tolist())

'[CLS] m a s [MASK] k r p s q r h g s [MASK] y l a t a s t m d h a r h g f l p [MASK] h r [MASK] t g i l d s i g r f f g g [MASK] r g a p k r g s g [MASK] d s h h p [MASK] r t a h y g s l p q [SEP] m k [MASK] k k t i g a [MASK] a l t t m f [MASK] a v s a s a v e k [MASK] i t v t a s v d p v i d l l [MASK] a d g n a l p s a v k l a y s p a s k t f e [MASK] y r [MASK] dixon t q v h t n d [MASK] t k k [MASK] i v k [MASK] a d [MASK] p q l [MASK] [MASK] v l n s t v [MASK] m p i s v s w g g q v l s t t a k e f [MASK] a a titanium l g y s a s g v n g [MASK] s s s q e l v i s a a p k t a g t a p t a g n y s g v v s l v m t l g [MASK] [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] 

In [ ]:
training_args = TrainingArguments(
    output_dir="./model/bert-retrained",
    overwrite_output_dir=True,
    num_train_epochs=2,
    per_device_train_batch_size=8,
    save_steps=10000,
    save_total_limit=2,
    seed=1
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset
)

In [ ]:
trainer.train()

trainer.save_model("./model/bert-retrained")

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 26149
  Num Epochs = 2
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 6538


Step,Training Loss
500,3.878200
1000,3.253200
1500,3.239100
2000,3.233700
2500,3.188600
3000,3.166800
3500,3.094300
4000,3.062700
4500,3.033400
5000,3.027900




Training completed. Do not forget to share your model on huggingface.co/models =)


Saving model checkpoint to ./model/bert-retrained
Configuration saved in ./model/bert-retrained/config.json
Model weights saved in ./model/bert-retrained/pytorch_model.bin


## Loading pre-trained model

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
import torch
from transformers import TrainingArguments, Trainer
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import EarlyStoppingCallback

In [ ]:
# Create torch dataset
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels:
            item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])

In [ ]:
def preprocess_data(data):

    # Preprocess data
    X = list(data["sequence"])
    y = list(data["label"])
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2)
    X_train_tokenized = tokenizer(X_train, padding=True, truncation=True, max_length=512)
    X_val_tokenized = tokenizer(X_val, padding=True, truncation=True, max_length=512)

    train_dataset = Dataset(X_train_tokenized, y_train)
    val_dataset = Dataset(X_val_tokenized, y_val)
    return train_dataset, val_dataset

In [ ]:
from transformers import RobertaTokenizer, RobertaForSequenceClassification, BertTokenizer, BertForSequenceClassification

df_train = pd.read_csv("./input/data_train.csv")

sequence_formatted = []
for seq in df_train['sequence'].values:
  sequence_formatted.append(" ".join(seq))

data = pd.DataFrame({'sequence':sequence_formatted, 'label':df_train['label'].tolist()})

#data = df_train

data_op = data[:int(len(data)/2)]


# Define pretrained tokenizer and model
batch_size=8
model_name = 'bert-base-uncased'


tokenizer = BertTokenizer.from_pretrained(model_name)

model = BertForSequenceClassification.from_pretrained("./model/bert-retrained", num_labels=2)



train_dataset_op, val_dataset_op = preprocess_data(data_op)
train_dataset, val_dataset = preprocess_data(data)

# ----- 2. Fine-tune pretrained model -----#
# Define Trainer parameters
def compute_metrics(p):
    
    pred, labels = p
    pred = np.argmax(pred, axis=1)

    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    recall = recall_score(y_true=labels, y_pred=pred)
    precision = precision_score(y_true=labels, y_pred=pred)
    f1 = f1_score(y_true=labels, y_pred=pred)

    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}

# Define Trainer
args = TrainingArguments(
    f"{model_name}-finetuned-classification",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=5,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model='f1',
    optim="adamw_torch"
)

def model_init():
    return model

trainer = Trainer(
    model_init=model_init,
    args=args,
    train_dataset=train_dataset_op,
    eval_dataset=val_dataset_op,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)


best_run = trainer.hyperparameter_search(n_trials=10, direction="maximize")


loading file https://huggingface.co/bert-base-uncased/resolve/main/vocab.txt from cache at /root/.cache/huggingface/transformers/45c3f7a79a80e1cf0a489e5c62b43f173c15db47864303a55d623bb3c96f72a5.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99
loading file https://huggingface.co/bert-base-uncased/resolve/main/added_tokens.json from cache at None
loading file https://huggingface.co/bert-base-uncased/resolve/main/special_tokens_map.json from cache at None
loading file https://huggingface.co/bert-base-uncased/resolve/main/tokenizer_config.json from cache at /root/.cache/huggingface/transformers/c1d7f0a763fb63861cc08553866f1fc3e5a6f4f07621be277452d26d71303b7e.20430bd8e10ef77a7d2977accefe796051e01bc2fc4aa146bc862997a1a15e79
loading configuration file https://huggingface.co/bert-base-uncased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.37395cee442ab11005bcd270f3c34464dc1704b715b5d7

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,No log,0.547364,0.674699,0.736842,0.388889,0.509091
2,No log,0.508663,0.783133,0.750000,0.750000,0.750000


***** Running Evaluation *****
  Num examples = 83
  Batch size = 8
Saving model checkpoint to bert-base-uncased-finetuned-classification/run-0/checkpoint-42
Configuration saved in bert-base-uncased-finetuned-classification/run-0/checkpoint-42/config.json
Model weights saved in bert-base-uncased-finetuned-classification/run-0/checkpoint-42/pytorch_model.bin
tokenizer config file saved in bert-base-uncased-finetuned-classification/run-0/checkpoint-42/tokenizer_config.json
Special tokens file saved in bert-base-uncased-finetuned-classification/run-0/checkpoint-42/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 83
  Batch size = 8
Saving model checkpoint to bert-base-uncased-finetuned-classification/run-0/checkpoint-84
Configuration saved in bert-base-uncased-finetuned-classification/run-0/checkpoint-84/config.json
Model weights saved in bert-base-uncased-finetuned-classification/run-0/checkpoint-84/pytorch_model.bin
tokenizer config file saved in bert-base-uncased

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,No log,0.504875,0.795181,0.756757,0.777778,0.767123
2,No log,0.500621,0.807229,0.777778,0.777778,0.777778


***** Running Evaluation *****
  Num examples = 83
  Batch size = 8
Saving model checkpoint to bert-base-uncased-finetuned-classification/run-1/checkpoint-42
Configuration saved in bert-base-uncased-finetuned-classification/run-1/checkpoint-42/config.json
Model weights saved in bert-base-uncased-finetuned-classification/run-1/checkpoint-42/pytorch_model.bin
tokenizer config file saved in bert-base-uncased-finetuned-classification/run-1/checkpoint-42/tokenizer_config.json
Special tokens file saved in bert-base-uncased-finetuned-classification/run-1/checkpoint-42/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 83
  Batch size = 8
Saving model checkpoint to bert-base-uncased-finetuned-classification/run-1/checkpoint-84
Configuration saved in bert-base-uncased-finetuned-classification/run-1/checkpoint-84/config.json
Model weights saved in bert-base-uncased-finetuned-classification/run-1/checkpoint-84/pytorch_model.bin
tokenizer config file saved in bert-base-uncased

...

***** Running Evaluation *****
  Num examples = 83
  Batch size = 8
[I 2022-07-19 15:58:30,371] Trial 7 pruned. 
Trial:
***** Running training *****
  Num examples = 330
  Num Epochs = 4
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 168


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,No log,0.679281,0.855422,0.785714,0.916667,0.846154
2,No log,0.841690,0.819277,0.838710,0.722222,0.776119
3,No log,0.812003,0.819277,0.769231,0.833333,0.800000
4,No log,0.811638,0.819277,0.769231,0.833333,0.800000


***** Running Evaluation *****
  Num examples = 83
  Batch size = 8
Saving model checkpoint to bert-base-uncased-finetuned-classification/run-8/checkpoint-42
Configuration saved in bert-base-uncased-finetuned-classification/run-8/checkpoint-42/config.json
Model weights saved in bert-base-uncased-finetuned-classification/run-8/checkpoint-42/pytorch_model.bin
tokenizer config file saved in bert-base-uncased-finetuned-classification/run-8/checkpoint-42/tokenizer_config.json
Special tokens file saved in bert-base-uncased-finetuned-classification/run-8/checkpoint-42/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 83
  Batch size = 8
Saving model checkpoint to bert-base-uncased-finetuned-classification/run-8/checkpoint-84
Configuration saved in bert-base-uncased-finetuned-classification/run-8/checkpoint-84/config.json
Model weights saved in bert-base-uncased-finetuned-classification/run-8/checkpoint-84/pytorch_model.bin
tokenizer config file saved in bert-base-uncased

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,No log,0.741790,0.807229,0.763158,0.805556,0.783784


***** Running Evaluation *****
  Num examples = 83
  Batch size = 8
[I 2022-07-19 16:00:38,646] Trial 9 pruned. 


In [ ]:
for n, v in best_run.hyperparameters.items():
    setattr(trainer.args, n, v)

trainer.train_dataset=train_dataset
trainer.eval_dataset=val_dataset

trainer.train()

***** Running training *****
  Num examples = 660
  Num Epochs = 5
  Instantaneous batch size per device = 64
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 1
  Total optimization steps = 415


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,No log,0.302826,0.909639,0.879310,0.864407,0.871795
2,No log,0.324948,0.909639,0.907407,0.830508,0.867257
3,No log,0.368140,0.909639,0.866667,0.881356,0.873950
4,No log,0.433276,0.873494,0.787879,0.881356,0.832000
5,No log,0.328801,0.921687,0.910714,0.864407,0.886957


***** Running Evaluation *****
  Num examples = 166
  Batch size = 8
Saving model checkpoint to bert-base-uncased-finetuned-classification/checkpoint-83
Configuration saved in bert-base-uncased-finetuned-classification/checkpoint-83/config.json
Model weights saved in bert-base-uncased-finetuned-classification/checkpoint-83/pytorch_model.bin
tokenizer config file saved in bert-base-uncased-finetuned-classification/checkpoint-83/tokenizer_config.json
Special tokens file saved in bert-base-uncased-finetuned-classification/checkpoint-83/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 166
  Batch size = 8
Saving model checkpoint to bert-base-uncased-finetuned-classification/checkpoint-166
Configuration saved in bert-base-uncased-finetuned-classification/checkpoint-166/config.json
Model weights saved in bert-base-uncased-finetuned-classification/checkpoint-166/pytorch_model.bin
tokenizer config file saved in bert-base-uncased-finetuned-classification/checkpoint-166/to

TrainOutput(global_step=415, training_loss=0.316370511342244, metrics={'train_runtime': 226.3049, 'train_samples_per_second': 14.582, 'train_steps_per_second': 1.834, 'total_flos': 813321494330400.0, 'train_loss': 0.316370511342244, 'epoch': 5.0})

In [ ]:
# Load trained model
model_path = "bert-base-uncased-finetuned-classification/checkpoint-83"
model = BertForSequenceClassification.from_pretrained(model_path, num_labels=2)

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', model_max_length=512, truncation=True)


# Define test trainer
trainer = Trainer(model)

loading configuration file bert-base-uncased-finetuned-classification/checkpoint-83/config.json
Model config BertConfig {
  "_name_or_path": "./model/bert-retrained",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "torch_dtype": "float32",
  "transformers_version": "4.20.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file bert-base-uncased-finetuned-classification/checkpoint-83/pytorch_model.bin
All model checkpoint weights were used when initializing BertForSequenceClassification

In [ ]:
import pandas as pd
# ----- 3. Predict -----#
# Load test data
#test_data = pd.read_csv("test.csv")
test = pd.read_csv("./input/data_test.csv")

sequence_formatted = []
for seq in test['sequence'].values:
  sequence_formatted.append(" ".join(seq))

test_data = pd.DataFrame({'sequence':sequence_formatted, 'label':test['label'].tolist()})


X_test = list(test_data["sequence"])
X_test_tokenized = tokenizer(X_test, padding=True, truncation=True, max_length=512)

# Create torch dataset
test_dataset = Dataset(X_test_tokenized)

# Make prediction
raw_pred, _, _ = trainer.predict(test_dataset)

# Preprocess raw predictions
y_pred = np.argmax(raw_pred, axis=1)


***** Running Prediction *****
  Num examples = 207
  Batch size = 8


In [ ]:
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import roc_auc_score

print("ROC_AUC:", roc_auc_score(test_data['label'], y_pred))

print(classification_report(test_data['label'], y_pred))

ROC_AUC: 0.8927616501145912
              precision    recall  f1-score   support

           0       0.91      0.90      0.91       119
           1       0.87      0.89      0.88        88

    accuracy                           0.89       207
   macro avg       0.89      0.89      0.89       207
weighted avg       0.89      0.89      0.89       207

